In [13]:
import pandas as pd
import numpy as np
from coinapi_v1 import CoinAPIv1
import datetime
from datetime import datetime, timedelta
import time
from time import mktime as mktime
import calendar
import json
import urllib.request


test_key = 'EED0F746-36FB-4CC4-8E7C-527333DFA6FB'

api = CoinAPIv1(test_key)


In [14]:
#this function computes how many seconds from now till midnight.
#the correction is due to the fact that some times are not utc by default.
#This can be checked transforming output into utc time.
def until_midnight():
    tomorrow = datetime.now() + timedelta(1)
    midnight = datetime(year=tomorrow.year, month=tomorrow.month, 
                        day=tomorrow.day, hour=0, minute=0, second=0)
    wrong_number=(midnight - datetime.now()).seconds
    return (wrong_number - 14399)

In [15]:
#Check whether list of assets is already available.
#If not, download it.
def get_assets():
    try:
        with open('assets.txt') as json_file:  
            assets= json.load(json_file)
    except:
        assets = api.metadata_list_assets()
        with open('assets.txt', 'w') as outfile:  
            json.dump(assets, outfile)  
    return(assets)

In [16]:
#Check whether list of symbols is already available.
#If not, download it.
def get_symbols():
    try:
        with open('symbols.txt') as json_file:  
            symbols = json.load(json_file)
    except:
        symbols = api.metadata_list_symbols()
        with open('symbols.txt', 'w') as outfile:  
            json.dump(symbols, outfile)  
    return(symbols)

In [17]:
#this function sends a request to the coinAPI server.
#If an error returns, it pauses until midnight to send 
#one more request (daily limit is hit).
#which = trades or quotes.
def request_exchange_rates(unix_time, base, quote):
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    while True:
        try:
            exchange=api.exchange_rates_get_specific_rate(base, quote, {'time': utctime})
        except urllib.error.HTTPError:
            time.sleep(until_midnight())
    return(exchange)

In [20]:
#This function checks whether exchange rates have already been downloaded.
#If not, it downloads it.
def get_exchange_rates(unix_time, base, quote):
    path='/Users/federicobuonerba/Documents/Etale/etale_data/exchange_rates_data/'
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    try:
        with open(path+'exchange_rate_'+str(base)+'_'+str(quote)+'_'+str(unix_time)+'.txt') as json_file:  
            exchange = json.load(json_file)   
    except:
        #print('downloading...')
        exchange=api.exchange_rates_get_specific_rate(base, quote, {'time': utctime})
        with open(path+'exchange_rate_'+str(base)+'_'+str(quote)+'_'+str(unix_time)+'.txt', 'w') as outfile:  
            json.dump(exchange, outfile)    
    return(exchange)  

In [19]:
#this function sends a request to the coinAPI server.
#If an error returns, it pauses until midnight to send 
#one more request (daily limit is hit).
#which = trades or quotes.
def request_trades(unix_time, symbol):
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    while True:
        try:
            trades_historical_data=api.trades_historical_data(symbol['symbol_id'],{'time_start': utctime , 'limit':1})
            break
        except urllib.error.HTTPError:
            time.sleep(until_midnight())
    return(trades_historical_data)

In [9]:
#This function checks whether certain data has already been downloaded.
#If not, download it using the previous request function.

def get_trades(unix_time, symbol):
    
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    ######path='Users/federicobuonerba/Documents/Etale/etale_data/trades_data'
    try:
        with open('trades_historical_data_'+str(unix_time)+'.txt', 'r') as json_file:  
            trades_historical_data = json.load(json_file)   
    except:
        #print('downloading...')
        trades_historical_data=request_trades(unix_time, symbol)
        with open('trades_historical_data_'+str(unix_time)+'.txt', 'w') as outfile:  
                json.dump(trades_historical_data, outfile)
    return(trades_historical_data)  

In [4]:
def make_df(unix_time, symbol):
    
    utctime = datetime.utcfromtimestamp(unix_time).strftime('%Y-%m-%dT%H:%M:%S')
    trades_historical_data = get_trades(unix_time, symbol)
    if trades_historical_data==[]:
        df=pd.DataFrame()
    else:   
        df=pd.DataFrame(data=trades_historical_data)
        df=df.drop(['time_coinapi'], axis=1)
        df=df.drop(['symbol_id'], axis=1)
        df['unix_time']=unix_time
        df['human_time']=utctime
        df['asset_base']=symbol['asset_id_base']
        df['asset_quote']=symbol['asset_id_quote']
        df['exchange_id']=symbol['exchange_id']
        df['type']=symbol['symbol_type']
        df['actual_time_exchange']=df['time_exchange']
        df=df.drop(['time_exchange'],axis=1)
        col=df.columns
        col1=col[4:6].append(col[0:4]).append(col[6:])
        df=df[col1]
    
    return(df)

In [5]:
#####Example#####
symbols=get_symbols()
unix_time= 1451606400
make_df(unix_time, symbols[8])

,unix_time,human_time,price,size,taker_side,uuid,asset_base,asset_quote,exchange_id,type,actual_time_exchange
0,1451606400,2016-01-01T00:00:00,0.041513,2.419,BUY,a1205a9c-9388-4898-af89-8b8d1e47cd75,ETH,BTC,BINANCE,SPOT,2017-12-18T20:54:34.1660000Z


In [6]:
#this function loops make_df over all possible symbols
def get_data_symbols(unix_time):
    df = pd.DataFrame()
    for symbol in symbols:
        df_new=make_df(unix_time, symbol)
        df=pd.concat([df, df_new])
    return(df)

In [ ]:
#this function loops get_data_symbols for the day starting at unix_time,
#sampled every hour. There are 1440 minutes in a day.
def get_hourly_data_day(unix_time, interval):
    df=pd.DataFrame()
    for t in range(24):
        df_new=get_data_symbols(unix_time + 3600*t)
        df=pd.concat([df, df_new])
    return(df)    
        

In [7]:
#this function loops get_data_symbols for the day starting at unix_time,
#sampled every minute. There are 1440 minutes in a day.
def get_minutely_data_day(unix_time, interval):
    df=pd.DataFrame()
    for t in range(1440):
        df_new=get_data_symbols(unix_time + 60*t)
        df=pd.concat([df, df_new])
    return(df)    
        
    

In [31]:
#this function creates a csv of the daily dataframe constructed with get_data_time
def create_csv(unix_time):
    df=get_daily_data(unix_time)
    df.to_csv('trades_historical_data_'+str(unix_time)+'.csv', index=False)